In [1]:
import json
import time
import pickle
import requests
import pandas as pd
from sklearn import model_selection, metrics

### Set sevice coordinates

In [2]:
url = 'http://localhost:5000/'

### Preprocess Data

In [3]:
df = pd.read_csv('data/train.csv')  # For example i will use Titanic dataset
df = df.set_index('PassengerId')

# Create dummy features
df['IsMale'] = pd.get_dummies(df['Sex'])['male'] 

# fill missing data
df['Age'] = df['Age'].fillna(df['Age'].median())

### Create request body

In [4]:
target = 'Survived'
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'IsMale']

df_train, df_test = model_selection.train_test_split(df)

data = json.dumps({'metric': 'accuracy', 'data': df_train.to_json(), 'features': features, 'target': target})

### Train model

In [5]:
# Send post request to start_calculation
r = requests.post(f'{url}/start_classification', data)
model_id = r.json()['model_id'] 

# wait while model is being trained
while r.status_code != 200:
    time.sleep(5)
    r = requests.get(f'{url}/get_model', params={'model_id': model_id})

# load model from binary file
model = pickle.loads(r.content)

# You can check this model final score on test data
score = requests.get(f'{url}/get_score', params={'model_id': model_id})
print(score.text)

{"model_id":"da1e9a49-76f8-4d4f-8e63-25c019377f39","score":0.8323353293413174,"score_type":"accuracy"}



### Use this model

In [6]:
metrics.accuracy_score(df_test[target], model.predict(df_test[features]))

0.8161434977578476